In [1]:
import numpy as np

# Get input:
This function gets an integer, then get the requests and stock length for that case.

In [2]:
def Get_input(i):
    # Open input file
    if (i == 1):
        infile = open('input1.stock', 'r')
    elif (i == 2):
        infile = open('input2.stock', 'r')
    elif (i == 3):
        infile = open('input3.stock', 'r')
    else:
        infile = open('input4.stock', 'r')

    # Read instance header for the pr1002
    length = int(infile.readline().strip().split()[2]) #length
    infile.readline()
    infile.readline()
    requests = np.array(list(map(int, infile.readline().strip().split(', ')[:-1]))) #requests

    # Close input file
    infile.close()
    
    return length, requests

# Cost function:
This function calculates the cost of the given order. The given order, determines the indexex of cuts one by one.

This is how to calculate cost:
1. Until all the requests have been read:
2. Add the request to last stock.
3. If the sum of requests got more than stock length, subtract the request from sum.
4. Add one to used stock number.

In [3]:
def calculate_cost(order, stock_length, requests):
    cost = 0
    s = 0
    i = 0
    while(s <= stock_length and i < len(order)):
        # Add the request
        s += requests[order[i]]
        if (s > stock_length):
            # Subtract the request and add one to number of stocks
            s -= requests[order[i]]
            i -= 1
            cost += 1
            s = 0
        i += 1
    return cost

# Selection:
We use tournament selection:
1. Find 4 random orders.
2. return the index of lowest cost.

In [4]:
def tournament_selection(costs):
    min_cost = np.inf
    best_index = 0
    
    # Find the lowest cost between for random orders
    for i in range(4):
        index = np.random.randint(len(costs))
        cost = costs[index]
        
        # Update the best index
        if cost < min_cost:
            min_cost = cost
            best_index = index
            
    return best_index

# Crossover:
Creating two new order, from given orders.
1. Get two random numbers named index1 and index2
2. make sure $index1 < index2$
3. create n_order1 and n_onder2 of n zeros
4. copy order1 and order2 from index1 to index2 to n_order1 and n_order2 respectively.
5. then from index2 trought index2, add anything from order1(order2) to n_order2(n_order1) if it hasn't been copied yet.

In [5]:
def crossover(order1, order2, n):
    # Random numbers
    index1 = np.random.randint(n)
    index2 = np.random.randint(n)
    if(index1 > index2):
        index1, index2 = index2, index1
    
    # copy from index1 to index2
    n_order1 = np.zeros(n, dtype='int')
    n_order2 = np.zeros(n, dtype='int')
    n_order1[index1:index2+1] = order1[index1:index2+1].copy()
    n_order2[index1:index2+1] = order2[index1:index2+1].copy()
    
    j = index2+1
    k = index2+1
    
    # copy the rest from the other parent, if has not been yet copied
    for i in range(n):
        if(not (order1[(index2+1+i)%n] in n_order2)):
            n_order2[j%n] = order1[(index2+1+i)%n]
            j = j+1
        if(not (order2[(index2+1+i)%n] in n_order1)):
            n_order1[k%n] = order2[(index2+1+i)%n]
            k = k+1
            
    return n_order1, n_order2

# Mutation:
change two random indexes.

In [6]:
def scramble_mutation(order):
    # Random number
    index1 = np.random.randint(len(order))
    index2 = np.random.randint(len(order))
        
    # Swap indexes
    t = order[index1].copy()
    order[index1] = order[index2].copy()
    order[index2] = t
    return order

# Stop condition:
there are 3 cases:
1. If the minimum cost is zero, then cut the running.
2. If the minimum cost is same as the last best cost. So add to count (in main function)
3. If the minimum cost is lower than last best. (update the best cost)

In [7]:
def stop_condition(costs, last_min):
    # Cut the running
    if(min(costs) == 0):
        last_min = 0
        return 0, last_min
    
    # last min hasn't changed
    elif(min(costs) == last_min):
        return 0.5, last_min
    
    # Update the lowest cost
    elif(min(costs) < last_min):
        last_min = min(costs)
        return 1, last_min

Using this function, we create the first population of different permutation from 1 to n.

In [8]:
def data_creation(n, population):
    # The population
    data = np.zeros((population, n), dtype='int')
    for i in range(population):
        # A random permutation
        data[i] = np.random.permutation(n)
        
    return np.array(data)

# Genetic Algorithm:
This is the main algorithm for Genetic:
1. Each time, we have population(data), and their costs.
2. First of all we sort the population.
3. Then Create children using crossover and mutation from selected parents.
4. Then sort the childen.
5. And at last, Update the population(Generation) with best children.
6. We do this until the stop condition is true.

In [18]:
def genetic_algorithm(inp, n_pop, iteration, s, replacement_rate, mu_chance):
    # Get the input
    stock_length, requests = Get_input(inp)
    chromosome_len = len(requests)
    
    # res_min and res_best_per to save the best orders
    res_min = np.array([], dtype='int')
    res_best_per = np.empty((0, chromosome_len), int)
    
    for _ in range(s):
        count = 0
        last_min = np.inf

        # Create the initial population
        data = data_creation(chromosome_len, n_pop)
        costs = np.zeros(n_pop, dtype='int')
        
        # Calculate costs
        for i in range(n_pop):
            costs[i] = calculate_cost(data[i], stock_length, requests)

        while(count < iteration):
            # Check for stop condition
            stop, last_min = stop_condition(costs, last_min)
            if not stop:
                break
            # If the best cost hasn't changed
            if stop == 0.5:
                count += 1
            else:
                count = 0
                
            #---------------  Sort the population based on their costs ---------------   
            # Sort costs     
            sorted_cost = np.sort(costs)
            sorted_data = np.empty((0, chromosome_len), int)

            # Create new numpy array to store the sorted data
            for i in range(n_pop):
                for index in np.where(costs == sorted_cost[i])[0]:
                    sorted_data = np.append(sorted_data, np.array([data[index].copy()]), axis=0)
                    i += 1
            data = sorted_data.copy()
            costs = sorted_cost.copy()
            
            #----------  Create n_pop chilren using crossover and mutation ----------- 
            children = np.empty((0, chromosome_len), int)
            for i in range(int(n_pop/2)):
                arr_index = []
                child = []
                # Randomly get two order from the population
                for i in range(2):
                    arr_index.append(tournament_selection(costs))
                    
                chance = np.random.rand()
                # with 0.95 chance, Create new orders using crossover
                if(chance <= 0.95):
                    child1, child2 = crossover(data[arr_index[0]], data[arr_index[1]], chromosome_len)
                    child.append(child1)
                    child.append(child2)
                # Otherwise, use the chosen orders themself
                else:
                    child1, child2 = data[arr_index[0]], data[arr_index[1]]
                    child.append(child1)
                    child.append(child2)

                chance = np.random.rand()
                # with mu_chance chance, use mutation to mutate the created children
                if(chance <= mu_chance):
                    child[0], child[1] = scramble_mutation(child[0]), scramble_mutation(child[1])
                children = np.append(children, np.array([child[0]]), axis=0)
                children = np.append(children, np.array([child[1]]), axis=0)
                
            #----------------  Sort the children based on their costs ---------------- 
            pop_child = len(children) #(G)
            cost_children = np.array([], dtype = 'int')
            
            # Calculate chilred costs
            for i in range(pop_child):
                cost_children = np.append(cost_children, calculate_cost(children[i], stock_length, requests))

            sorted_cost_children = np.sort(cost_children)
            sorted_children = np.empty((0, chromosome_len), int)
            
            # Sort the children, as we did for population
            for i in range(pop_child):
                for index in np.where(cost_children == sorted_cost_children[i])[0]:
                    sorted_children = np.append(sorted_children, np.array([children[index].copy()]), axis=0)
                    i += 1
            children = sorted_children.copy()
            cost_children = sorted_cost_children.copy()
            
            #-------------  Updating the population(Generation) and costs -------------
            # Update the population with best of population and best children
            for i in range(int(replacement_rate*(n_pop))):
                data[-1*(i+1)] = children[i].copy()
                costs[-1*(i+1)] = cost_children[i].copy()

        res_min = np.append(res_min, last_min)
        res_best_per = np.append(res_best_per, np.array([data[np.where(costs == min(costs))[0][0]]]), axis=0)

    return res_min, res_best_per

Results for input1. As we can see the lowest cost is 54 and has the given order of cutting.

The order given, is based on the data in input1, which means that the numbers are indexes for original data.

In [14]:
res_min, res_per = genetic_algorithm(1, 50, 1000, 5, 0.5, 0.5)
print(res_min)
print(res_per[np.where(res_min == min(res_min))[0][0]])

[54 55 56 55 56]
[  8  27  41  49  80   9  47   7  17  33  15   3  44 127  91  28 131  46
 118 128 113 116  23  69 101   4 103  29 129   5  86  83  99   1 133  19
  38  34  20 124  98  95  73  89 104  78  14  50 130  54  42 110 125  45
  71  96  84  37  32  40 122 106  25  11 107  97  79  24  93 109  64  87
 132  90  85  18  59  35   6 102  51  36 119  62  13  60 138  30  22  94
  10  81 120 105  12 100 112  72  77  57  56  31  43  58 136 115  63 134
  88  61 126 135   0   2 111  74  21  68  67  92 108 114  26 137  66  48
 117  82  16  52  55 121 123  53  65  70  76  39  75]


Results for input2. As we can see the lowest cost is 83 and has the given order of cutting.

The order given, is based on the data in input2, which means that the numbers are indexes for original data.

In [15]:
res_min, res_per = genetic_algorithm(2, 50, 1000, 5, 0.5, 0.5)
print(res_min)
print(res_per[np.where(res_min == min(res_min))[0][0]])

[84 84 83 83 83]
[ 48  20  59  56 138 124  35  53  58 198 176 115  57  54  31  72 157  99
  81 144 179  18  64 163  77 139 126 210  94 129 208  17  87  26 174 122
 113 134   6  91 209 104  97 110  88 184 206  38  11  36  14  62 217 181
 185 195  98 168 132  68  63 121  49  73 183  78  25  61 165 130  13   7
  28   5 135 215 169  10  19 212  24 155   1  29 200  22 164 201  69  82
 213  34  70 197 203 133  80 128  65 152  15 131 170  30  52  50 150  43
   0  27 190 214 102 193   9 192 189 149 191 101 143  93  55  32  86 114
  83  96  16  60 159   8 146 180 127 194 187   2  41 137  21  92  47  89
   3  75 116 199 120  42 186 119 100  95  12 111  37  76 171 216 109 161
 107  66 172  44 158 108 175 178 173 160 207 166  71 148  67 177 118 147
 125 136  90 154 156  46  39  33 202 182 167 112  23 117  84 204   4 140
 196 153 103  40 123  79 188 211 151 162  45 142 141  74 145 205 106 105
  51  85]


Results for input3. As we can see the lowest cost is 105 and has the given order of cutting.

The order given, is based on the data in input3, which means that the numbers are indexes for original data.

In [16]:
res_min, res_per = genetic_algorithm(3, 50, 1000, 5, 0.5, 0.5)
print(res_min)
print(res_per[np.where(res_min == min(res_min))[0][0]])

[107 108 105 106 108]
[579 474 494  19 559 246 225 305 521 528  41 525 664 750 311 216 824  45
 315 739 477 248 832 697 842  47 229 242 858 754 169 529 357 159 191 274
 296 280 523 199 744 581 756 131 600 511 686 170 177 356  83 527 213 220
 379 277 115 141 349 219 693 104 314 815 347 481 150 526 767 463 696 105
 338 713  55 224 255 358 586 689 667 108 256 125 289 770 397 786 223 572
  77 272 595 764 292 692 124 775 516 265 565 835 189 607 344 577 561 682
 627  22 839  78 161 798  33 691 293 384 574 631 562 520 553 657  46 377
 571 837  97 227 425 547 795  69 454 261 643 343 102 455 610 578  37 680
 466 656 472  49 706 637 146 208 390  68 508 809 158 113 545 734  64 430
 592 532 442 668 649 326  71 251 276  58 852 164 298 613 231 856 836 540
 820 471 781 800 715  75 727 428 557 544 354 558 464  94 187 185 807 735
 826 337 419 658 507 235 647 708 443 333 290 182 381  80  96 429 819 711
  30  59 633 413 399 564 408 136 628 844 602 695 103 776 738 393 741 765
 406 192 380 369 671 200 226 

Results for input4. As we can see the lowest cost is 243 and has the given order of cutting.

The order given, is based on the data in input3, which means that the numbers are indexes for original data.

In [17]:
res_min, res_per = genetic_algorithm(4, 50, 500, 5, 0.5, 0.5)
print(res_min)
print(res_per[np.where(res_min == min(res_min))[0][0]])

[245 245 243 244 243]
[662 324 600 558 176  63 538 867 344 698 379 158 601 718 348 180 546 218
 811 198 454 455 199 850  78 881 589 710 525 653 760  61  54 632 233  20
 261 215 341 512 470 478 395 741 249 578 356 133 206 445 123  12 269 166
 407 224 720 734 167 212 842 110 893 410 208 244 294 500 807 453 115 393
 839 489 780 234  44 868 296 774 300  94 162 692 526 569 613 590 800 645
 458 877 373 751 161 353 295 663 165  68 408 495 104 448 361 853 388 895
 494 593 821 671 628  84 779  56 417 776 141 783 825 638 301 611 426 749
 550 560 861 829 882  23 573 465   2 140 306 333  98   9 896 358 818 420
 334 880 331 442 886 837 878  47 510 332 149 828 351 713  60 580 131 226
 320 790 824 685 804 485 371 483 153 461 887 181 375 266 307 326 288 187
 687 390 391 365 210 769 464 265 524 128 112 189 159 387 397 229 553 587
 188 866 834 752 816 708 717 380 472   4 570 316 583 245  22 404  64 441
 598 772  11 264 325 665 615 657 377 622 678  86 809  70 481 872 289 429
 588 256 272  65 786 631 854 